# 1、导入相关依赖包

In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import os
import json

# 自动下载模型时，指定使用modelscope; 否则，会从HuggingFace下载
os.environ['VLLM_USE_MODELSCOPE']='True'

INFO 02-23 15:16:41 __init__.py:183] Automatically detected platform cuda.


# 2.定义LLM处理函数

In [2]:
def get_completion(prompts, model, tokenizer=None, max_tokens=8192, temperature=0.6, top_p=0.95, max_model_len=2048):
    stop_token_ids = [151329, 151336, 151338]
    # 创建采样参数。temperature 控制生成文本的多样性，top_p 控制核心采样的概率
    sampling_params = SamplingParams(temperature=temperature, top_p=top_p, max_tokens=max_tokens, stop_token_ids=stop_token_ids)
    # 初始化 vLLM 推理引擎
    llm = LLM(model=model, tokenizer=tokenizer, max_model_len=max_model_len,trust_remote_code=True)
    outputs = llm.generate(prompts, sampling_params)
    return outputs

# 3.指定模型地址&定义消息

In [3]:
# 初始化 vLLM 推理引擎
model='/root/autodl-tmp/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B' # 指定模型路径
# model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" # 指定模型名称，自动下载模型
tokenizer = None
# 加载分词器后传入vLLM 模型，但不是必要的。
# tokenizer = AutoTokenizer.from_pretrained(model, use_fast=False) 

text = ["请帮我推荐一种深圳到北京的出行方式<think>\n", ] 
# 可用 List 同时传入多个 prompt，根据 DeepSeek 官方的建议，每个 prompt 都需要以 <think>\n 结尾，
#如果是数学推理内容，建议包含（中英文皆可）：Please reason step by step, and put your final answer within \boxed{}.

# messages = [
#     {"role": "user", "content": prompt+"<think>\n"}
# ]
# 作为聊天模板的消息，不是必要的。
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )


# 4、获取输出结果

In [4]:
# 思考需要输出更多的 Token 数，max_tokens 设为 8K，根据 DeepSeek 官方的建议，temperature应在 0.5-0.7，推荐 0.6
outputs = get_completion(text, model, tokenizer=tokenizer, max_tokens=8192, temperature=0.6, top_p=0.95, max_model_len=2048) 

# 输出是一个包含 prompt、生成文本和其他信息的 RequestOutput 对象列表。
# 打印输出。
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    if r"</think>" in generated_text:
        think_content, answer_content = generated_text.split(r"</think>")
    else:
        think_content = ""
        answer_content = generated_text
    print(f"Prompt: {prompt!r}, Think: {think_content!r}, Answer: {answer_content!r}")

INFO 02-23 15:19:09 config.py:526] This model supports multiple tasks: {'score', 'classify', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 02-23 15:19:09 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='/root/autodl-tmp/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', speculative_config=None, tokenizer='/root/autodl-tmp/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False),

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 02-23 15:19:14 model_runner.py:1116] Loading model weights took 14.2717 GB
INFO 02-23 15:19:15 worker.py:266] Memory profiling takes 0.83 seconds
INFO 02-23 15:19:15 worker.py:266] the current vLLM instance can use total_gpu_memory (23.68GiB) x gpu_memory_utilization (0.90) = 21.32GiB
INFO 02-23 15:19:15 worker.py:266] model weights take 14.27GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 5.59GiB.
INFO 02-23 15:19:15 executor_base.py:108] # CUDA blocks: 6536, # CPU blocks: 4681
INFO 02-23 15:19:15 executor_base.py:113] Maximum concurrency for 2048 tokens per request: 51.06x
INFO 02-23 15:19:17 model_runner.py:1435] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:17<00:00,  2.00it/s]

INFO 02-23 15:19:34 model_runner.py:1563] Graph capturing finished in 18 secs, took 0.21 GiB
INFO 02-23 15:19:34 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 20.82 seconds



Processed prompts: 100%|██████████| 1/1 [00:20<00:00, 20.83s/it, est. speed input: 0.62 toks/s, output: 49.26 toks/s]

Prompt: '请帮我推荐一种深圳到北京的出行方式<think>\n', Think: '好的，我现在要帮用户推荐从深圳到北京的出行方式。用户之前已经问过这个问题，现在可能需要进一步的信息或不同的选项。首先，我得考虑用户的需求可能是什么。他们可能时间紧迫，或者喜欢不同的旅行风格。\n\n深圳到北京的路线有很多，比如高铁、飞机、大巴，还有可能的自驾或者中转其他城市。我应该先列出几种主要的交通方式，然后详细说明每种方式的优缺点，这样用户可以根据自己的情况选择。\n\n首先，高铁是最快的方式，时间大概4-5小时，而且舒适，还有直达的班次。但高铁需要提前预订，特别是节假日，可能要提前一到两周。高铁适合大多数用户，尤其是那些时间紧张或喜欢高铁舒适环境的人。\n\n然后是飞机，深圳有机场，北京有首都机场和大兴机场。飞机时间大约2-3小时，但价格比较高，而且航班可能比较紧张，尤其是热门季节。如果用户喜欢 direct flight，或者时间不够，飞机是个不错的选择。\n\n接下来是大巴，虽然时间较长，但价格便宜，而且在车上可以睡觉，体验更像旅行。但大巴的班次可能不如高铁频繁，而且在大站之间可能比较慢，适合预算有限的用户。\n\n另外，用户可能还考虑是否需要中转。比如从深圳飞到上海，再转到北京，或者深圳到天津再到北京。这样的路线可能在价格上更有优势，但时间会更长，可能不太适合时间紧张的用户。\n\n如果用户喜欢自驾，可以考虑从深圳到北京自驾，但需要考虑时间、路线、油费等因素，而且北京的限行政策可能需要提前了解。\n\n还有，是否有直达的高铁？比如深圳北站到北京西站的G字头列车，这可能是一个快速的选择。如果用户对高铁比较熟悉，这样的选择会很方便。\n\n我还需要提醒用户注意一些细节，比如高铁需要提前订票，尤其是节假日，可能会有 seat changes 的限制。飞机方面，提前预订机票，选择合适的航班，避免延误。\n\n另外，北京的天气可能在冬季较冷，用户可能需要带保暖衣物，或者考虑是否需要提前预订住宿。\n\n综合考虑，我应该推荐高铁作为首选，因为时间快、舒适，然后飞机作为替代，接着是大巴，最后是中转或自驾。这样用户可以根据自己的情况和预算做出选择。\n', Answer: '\n\n深圳到北京的出行方式多样，您可以根据自己的时间和预算选择最适合的方式。以下是几种主要的出行方式推荐